In [ ]:
import seaborn as sns
import datetime 
import numpy as np
from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import sklearn #(version == 1.2.2)
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import learning_curve
%matplotlib inline

In [ ]:
Data911 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\911Project\raw_datasets\911.csv")
Data911.head(5) 
#e - dummy variable

In [ ]:
CleanData911 = Data911.dropna()
CleanData911

In [ ]:
CleanData911.info()

In [ ]:
CleanData911.describe()

In [ ]:
Grouped911Calls = CleanData911.groupby("zip")["e"].count().reset_index(name="count")
Grouped911Calls = Grouped911Calls.sort_values(by="count",ascending = False)
Grouped911Calls.head(5)

In [ ]:
GroupedTownCalls = CleanData911.groupby("twp")["e"].count().reset_index(name="count")
GroupedTownCalls = GroupedTownCalls.sort_values(by="count",ascending = False)
GroupedTownCalls.head(5)

#### Unique number of 911 call titles

In [ ]:
len(CleanData911["title"].unique())

In [ ]:
CleanData911['EMT Reason'] = CleanData911['title'].str.split(':').str[0]
CleanData911

In [ ]:
CleanData911['EMT Reason2'] = CleanData911.apply(lambda row : row['title'].split(':')[0],axis = 1)
CleanData911 = CleanData911.drop("EMT Reason2",axis = 1)
CleanData911.groupby("EMT Reason")["e"].count()#reset_index(name="count")

In [ ]:
# Create a countplot (bar plot of category counts)
fig = px.histogram(CleanData911, x="EMT Reason", title='Count of 911 Calls by EMT Reason', 
                   labels={'category': "EMT Reason", 'count': 'Count'},
                   color='EMT Reason')

# Show the plot
fig.show()

In [ ]:
CleanData911['timeStamp'] = pd.to_datetime(CleanData911['timeStamp'], format = '%Y-%m-%d %H:%M:%S')
time = CleanData911['timeStamp'].iloc[0]
time.weekday()
CleanData911['Hour'] = CleanData911.apply(lambda row : row['timeStamp'].hour,axis = 1)
CleanData911['Month'] = CleanData911.apply(lambda row : row['timeStamp'].month,axis = 1)

In [ ]:
dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}
dmap

In [ ]:
CleanData911['Day of Week'] = CleanData911.apply(lambda row : row['timeStamp'].weekday(),axis = 1)
CleanData911['Day of Week'] = CleanData911['Day of Week'].map(dmap)

In [ ]:
# Create a countplot (bar plot of category counts)
fig = px.histogram(CleanData911, x="Day of Week", title='Countplot of Categories', 
                   labels={'category': "Day of Week", 'count': 'EMT Reason'},
                   color='EMT Reason')

# Show the plot
fig.show()

In [ ]:
# Create a countplot (bar plot of category counts)
fig = px.histogram(CleanData911, x="Month", title='Countplot of Categories', 
                   labels={'category': "Month", 'count': 'EMT Reason'},
                   color='EMT Reason')
fig.update_layout(bargap=.2)
# Show the plot
fig.show()

In [ ]:
# Create a countplot (bar plot of category counts)
fig = px.histogram(CleanData911, x="Day of Week", title='Countplot of Categories', 
                   labels={'category': "Day of Week", 'count': 'EMT Reason'},
                   color='EMT Reason')

# Show the plot
fig.show()

In [ ]:
GroupByMonth = CleanData911.groupby("Month")["e"].count()
GroupByMonthDF = GroupByMonth.to_frame()
Tester = pd.DataFrame({'Count': GroupByMonth})
GroupedByMonth = CleanData911.groupby("Month")["e"].count().reset_index(name="count")
GroupedByMonth = GroupedByMonth.sort_values(by="count",ascending = False)

In [ ]:
# Create scatter plot with regression line
fig = px.scatter(GroupedByMonth, x='Month', y='count', trendline='ols',
                 title='Scatter Plot with Linear Regression',
                 labels={'x': 'Month', 'y': 'Count'})

# Show the plot
fig.show()

In [ ]:
byMonth = CleanData911.groupby('Month').count()
Test = byMonth.reset_index()

In [ ]:
# Create scatter plot with regression line
fig = px.scatter(Test, x='Month', y='e', trendline='ols',
                 title='Scatter Plot with Linear Regression',
                 labels={'x': 'Month', 'y': 'e'})

# Show the plot
fig.show()

In [ ]:
CleanData911['Date'] = CleanData911.apply(lambda row : row['timeStamp'].date(),axis = 1)
byDate = CleanData911.groupby('Date').count()
TestDate = byDate.reset_index()

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=TestDate['Date'],
        y=TestDate['e'],
        mode='lines+markers',  # Equivalent to marker='o' and linestyle='-'
        name='Data',
        line=dict(color='blue', width=2),  # Equivalent to color='b'
        marker=dict(size=6)
    )
)

# Update layout
fig.update_layout(
    title='Time Series Plot',
    xaxis_title='Date',
    yaxis_title='e',
    xaxis=dict(
        tickformat='%b %d, %Y',
        tickangle=45,
        showgrid=True
    ),
    yaxis=dict(
        showgrid=True
    ),
    hovermode='x unified',
    template='plotly_white',
    showlegend=True
)

# Display the plot
fig.show()

In [ ]:
EMS = CleanData911[CleanData911['EMT Reason'] == 'EMS']
Fire = CleanData911[CleanData911['EMT Reason'] == 'Fire']
Traffic = CleanData911[CleanData911['EMT Reason'] == 'Traffic']

In [ ]:
byDateEMS = EMS.groupby('Date').count()
byDateFire = Fire.groupby('Date').count()
byDateTraffic = Traffic.groupby('Date').count()

In [ ]:
TestDateEMS = byDateEMS.reset_index()
TestDateFire = byDateFire.reset_index()
TestDateTraffic = byDateTraffic.reset_index()

In [ ]:
fig = go.Figure()

# EMS data
fig.add_trace(
    go.Scatter(
        x=TestDateEMS['Date'],
        y=TestDateEMS['e'],
        mode='lines+markers',
        name='EMS',
        line=dict(color='blue', width=2),  # Equivalent to color='b', linestyle='-'
        marker=dict(symbol='cross', size=8)  # Approximates marker='1' (tri-down)
    )
)

# Fire data
fig.add_trace(
    go.Scatter(
        x=TestDateFire['Date'],
        y=TestDateFire['e'],
        mode='lines+markers',
        name='Fire',
        line=dict(color='green', width=2),  # Equivalent to color='g', linestyle='-'
        marker=dict(symbol='x', size=8)  # Approximates marker='2' (tri-up)
    )
)

# Traffic data
fig.add_trace(
    go.Scatter(
        x=TestDateTraffic['Date'],
        y=TestDateTraffic['e'],
        mode='lines+markers',
        name='Traffic',
        line=dict(color='red', width=2),  # Equivalent to color='r', linestyle='-'
        marker=dict(symbol='diamond', size=8)  # Approximates marker='3' (tri-left)
    )
)

# Update layout
fig.update_layout(
    title='Time Series Comparison: EMS, Fire, and Traffic',
    xaxis_title='Date',
    yaxis_title='e',
    xaxis=dict(
        tickformat='%b %d, %Y',
        tickangle=45,
        showgrid=True
    ),
    yaxis=dict(
        showgrid=True
    ),
    hovermode='x unified',
    template='plotly_white',
    showlegend=True
)

# Display the plot
fig.show()